In [0]:
import sys

sys.path.append("../include_utils/")

#from IPython.parallel import Client
import ipyparallel as ipp
import os, time
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import pickle
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
import pysam
from collections import OrderedDict, namedtuple
import operator
import multiprocessing as mp
from ipyparallel import Client
from Bio import SeqIO
import pyfaidx
import pyfasta
import textwrap
import tempfile
%matplotlib inline
from collections import Counter

In [0]:
cd "~/eckertlab/Mitra/mapping/split_parallel/collapsed/"

In [0]:
mapped = !ls *mapped.bam

In [0]:
samtools = "/home/cfriedline/bin/samtools"

In [0]:
from subprocess import Popen, PIPE, STDOUT

In [0]:
rc = Client(profile = "sge")
dv = rc[:]
lv = rc.load_balanced_view()
len(dv)

In [0]:
with dv.sync_imports():
    from subprocess import Popen, PIPE, STDOUT
    import os
    import socket
    import numpy

In [0]:
bams = !ls *mapped.bam

In [0]:
f = pysam.AlignmentFile(bams[0])

In [0]:
for x in f.fetch():
    print(x.reference_start, x.reference_end, x.alen, x.mapping_quality)

In [0]:
dv['samtools'] = samtools

In [0]:
%px cd "~/eckertlab/Mitra/mapping/split_parallel/collapsed/"

In [0]:
def run_cmd(cmd):
    p = Popen(cmd, stdout=PIPE, shell=True)
    p.wait()
    stdout = list(p.stdout)
    return stdout

def get_pos_of_alignment(args):
    bam, contig = args
    cmd1 = "%s view %s | grep -n %s" % (samtools, bam, contig)
    cmd2 = "%s view %s | grep -c ." % (samtools, bam)
    print(cmd1)
    stdout1 = run_cmd(cmd1)
    print(cmd2)
    stdout2 = run_cmd(cmd2)
    total = int(stdout2[0].decode().strip())
    if len(stdout1) == 0:
        return bam, total   
    lines = []
    for elem in stdout1:
        elem = elem.decode().strip()
        elem = elem.split()
        line = elem[0].split(":")[0]
        lines.append(int(line))
    maxx = max(lines)
    return bam, maxx, total, maxx/total

In [0]:
dv['get_pos_of_alignment'] = get_pos_of_alignment
dv['run_cmd'] = run_cmd

In [0]:
contig = "super10388"
contig = "super10514"
jobs = []
for b in bams:
    j = lv.apply_async(get_pos_of_alignment, (b, contig))
    jobs.append(j)

In [0]:
np.sum([x.ready() for x in jobs])

In [0]:
percent_done = []
for j in jobs:
    if j.ready():
        res = j.r
        if len(res) == 4:
            percent_done.append(j[-1])
pd.Series(percent_done).describe()

### Progress 
```
count    357.000000
mean       0.098760
std        0.022912
min        0.028845
25%        0.085339
50%        0.104062
75%        0.117053
max        0.133072
dtype: float64
```

```
count    350.000000
mean       0.099311
std        0.023422
min        0.029170
25%        0.085592
50%        0.104524
75%        0.118051
max        0.134560
dtype: float64
```
